This is modifying the book's code to be more object-oriented and easy to follow.

I will maintain an explicit event list object which will always be sorted by time. Events will be objects subclassing the same base class.

The functions which are called for the various events will schedule various other events by inserting them into this event list.

In [40]:
from dataclasses import dataclass, field

@dataclass
class Event:
    type_: int
    time: float

    def __gt__(self, other) -> bool:
        return self.time > other.time

class EventList:
    def __init__(self):
        self._list = []

    def pop(self):
        """Pop the next event from the list."""
        if self._list:
            event, self._list = self._list[0], self._list[1:]
            return event
        else:
            return None
    
    def get(self):
        """Get the next event from the list."""
        if self._list:
            return self._list[0]
        else:
            return None
        
    def insert(self, event: Event):
        """Schedule an event by inserting it into the list."""
        if not self._list:
            self._list = [event]
        else:
            for i in range(len(self._list)):
                if self._list[i] > event:
                    self._list = self._list[:i] + [event] + self._list[i:]
                    return
            self._list = self._list + [event]
            return

@dataclass
class Queue:
    _list: float = field(default_factory=list) # arrival time

    def __len__(self):
        return len(self._list)
    
    def add(self, value):
        self._list.append(value)
    
    def pop(self):
        val = self._list[0]
        self._list = self._list[1:]
        return val
    
    def __bool__(self):
        return bool(self._list)

In [41]:
el = EventList()

e1 = Event(0, 1.23)
e2 = Event(0, 5.25)
e3 = Event(2, 3.21)

el.insert(e1)
el.insert(e2)
el.insert(e3)

print(el._list)

[Event(type_=0, time=1.23), Event(type_=2, time=3.21), Event(type_=0, time=5.25)]


In [50]:
import numpy as np
from enum import Enum

MEAN_INTERARRIVAL = 1.0
MEAN_SERVICE_1 = 0.7
MEAN_SERVICE_2 = 0.9

NUM_EVENTS = 4
SIM_TIME = 0.0

SERVER_STATUS = []
TIME_ARRIVAL_1 = [] # queue 1
TIME_ARRIVAL_2 = [] # queue 2
TIME_NEXT_EVENT = [] # event list
TIME_LAST_EVENT = 0

NUM_IN_Q_1 = 0
NUM_IN_Q_2 = 0
TOTAL_OF_DELAYS_1 = 0.0
TOTAL_OF_DELAYS_2 = 0.0
NUM_CUSTS_DELAYED_1 = 0
NUM_CUSTS_DELAYED_2 = 0
AREA_NUM_IN_Q_1 = 0.0
AREA_NUM_IN_Q_2 = 0.0
AREA_SERVER_STATUS_1 = 0.0
AREA_SERVER_STATUS_2 = 0.0

EVENT_LIST = EventList()
Q1 = Queue()
Q2 = Queue()

def expon(mean): 
    return np.random.exponential(scale=mean)

class ServiceStatus(Enum):
    IDLE = 0
    BUSY = 1

def initialize():
    """Initialize simulation variables."""
    # init sim clock
    global SIM_TIME
    SIM_TIME = 0.0

    # init state vars
    global SERVER_STATUS
    SERVER_STATUS = [ServiceStatus.IDLE, ServiceStatus.IDLE]
    global Q1, Q2
    Q1 = Queue()
    Q2 = Queue()

    # init event list
    global TIME_NEXT_EVENT, MEAN_INTERARRIVAL
    global EVENT_LIST
    e1 = Event(type_=0, time=SIM_TIME + expon(MEAN_INTERARRIVAL))
    e_term = Event(type_=3, time=1_000)
    EVENT_LIST.insert(e1)
    EVENT_LIST.insert(e_term)


def timing():
    """Compare the timings of the next of each possible event type to find
    the next event to occur. Set NEXT_EVENT_TYPE accordingly, and advance the
    simulation clock to the time of occurrence of this event.
    """
    global NEXT_EVENT_TYPE, NUM_EVENTS, SIM_TIME

    global EVENT_LIST
    
    next_event = EVENT_LIST.pop()
    NEXT_EVENT_TYPE = next_event.type_
    SIM_TIME = next_event.time


def arrive1():
    """Handle the arrival of a customer to Server 1.
    If the server is busy, the customer enters Queue 1.
    If the server is idle, the customer immediately enters Service and
        schedules its departure/arrival to Queue 2.
    """
    delay = 0

    # schedule the next arrive1 event
    global EVENT_LIST, SIM_TIME, MEAN_INTERARRIVAL
    next_arrive1 = Event(type_=0, time=SIM_TIME + expon(MEAN_INTERARRIVAL))
    EVENT_LIST.insert(next_arrive1)
    
    global SERVER_STATUS
    if SERVER_STATUS[0] is ServiceStatus.BUSY:
        # if S1 is busy, go to Queue 1
        # store arrival time of this customer at end of queue
        global Q1
        Q1.add(SIM_TIME)

    else:
        # if S1 is idle, go directly to service
        delay = 0.0
        global TOTAL_OF_DELAYS_1, NUM_CUSTS_DELAYED_1
        TOTAL_OF_DELAYS_1 += delay
        NUM_CUSTS_DELAYED_1 += 1

        # make Server 1 busy
        SERVER_STATUS[0] = ServiceStatus.BUSY

        # schedule the departure
        global MEAN_SERVICE_1
        arrive2_event = Event(type_=1, time=SIM_TIME+expon(MEAN_SERVICE_1))
        EVENT_LIST.insert(arrive2_event)


def arrive2():
    """Handle the arrival of a customer to Server 2.
    If the server is busy, the customer enters Queue 2.
    If the server is idle, the customer immediately enters Service and
        schedules its Departure.

    If Q1 is empty, S1 must be made idle.
    Otherwise, if Q1 is nonempty, the next customer begins service.
    """

    # handle the customer departing from S1
    global NUM_IN_Q_1, SERVER_STATUS, EVENT_LIST
    global Q1
    if not Q1:
        # Q1 is empty, so S1 becomes idle
        SERVER_STATUS[0] = ServiceStatus.IDLE

    else:
        # There are customers in Q1, so they start service
        # compute delay of customer starting service S1
        global SIM_TIME, TIME_ARRIVAL_1, TOTAL_OF_DELAYS_1
        time_arrival = Q1.pop() # move customers up in Q1
        delay = SIM_TIME - time_arrival
        TOTAL_OF_DELAYS_1 += delay

        # increment the number of customers delayed, schedule next departure
        global NUM_CUSTS_DELAYED_1, MEAN_SERVICE_1
        NUM_CUSTS_DELAYED_1 += 1
        depart_event = Event(type_=1, time=SIM_TIME + expon(MEAN_SERVICE_1))
        EVENT_LIST.insert(depart_event)

    
    # Handle customer arriving at S2
    delay = 0
    
    if SERVER_STATUS[1] is ServiceStatus.BUSY:
        # if S2 is busy, go to Queue 2
        # store arrival time of this customer at end of queue
        global Q2
        Q2.add(SIM_TIME)

    else:
        # if S2 is idle, go directly to service 2
        delay = 0.0
        global TOTAL_OF_DELAYS_2, NUM_CUSTS_DELAYED_2
        TOTAL_OF_DELAYS_2 += delay
        NUM_CUSTS_DELAYED_2 += 1

        # make server 2 busy
        SERVER_STATUS[1] = ServiceStatus.BUSY

        # schedule departure
        global MEAN_SERVICE_2
        depart_event = Event(type_=2, time=SIM_TIME + expon(MEAN_SERVICE_2))
        EVENT_LIST.insert(depart_event)


def depart():
    """Handle the departure of a customer from service 2.
    If Q2 is nonempty, the next customer's service is started and
        their departure is scheduled.
    """
    global Q2
    if not Q2:
        # Q2 is empty
        global SERVER_STATUS
        SERVER_STATUS[1] = ServiceStatus.IDLE

    else:
        # Q2 is nonempty
        # compute delay
        global SIM_TIME, TIME_ARRIVAL_2, TOTAL_OF_DELAYS_2
        time_arrival = Q2.pop()
        delay = SIM_TIME - time_arrival
        TOTAL_OF_DELAYS_2 += delay

        # increment number of customers delayed, schedule the departure
        global NUM_CUSTS_DELAYED_2, MEAN_SERVICE_2
        NUM_CUSTS_DELAYED_2 += 1
        global EVENT_LIST
        depart_event = Event(type_=2, time=SIM_TIME+expon(MEAN_SERVICE_2))
        EVENT_LIST.insert(depart_event)


def report():
    """Log statistics"""
    global TOTAL_OF_DELAYS_1, TOTAL_OF_DELAYS_2, NUM_CUSTS_DELAYED_1, NUM_CUSTS_DELAYED_2, AREA_NUM_IN_Q_1, AREA_NUM_IN_Q_2, SIM_TIME, AREA_SERVER_STATUS_1, AREA_SERVER_STATUS_2
    print(f"\n\nAverage delay in queue 1 {TOTAL_OF_DELAYS_1 / NUM_CUSTS_DELAYED_1:11.3f} minutes")
    print(f"Average number in queue 1 {AREA_NUM_IN_Q_1 / SIM_TIME:10.3f}")
    print(f"Server 1 utilization {AREA_SERVER_STATUS_1 / SIM_TIME:15.3f}")
    # print(f"Time simulation ended {SIM_TIME:15.3f} minutes")
    print(f"\nAverage delay in queue 2 {TOTAL_OF_DELAYS_2 / NUM_CUSTS_DELAYED_2:11.3f} minutes")
    print(f"Average number in queue 2 {AREA_NUM_IN_Q_2 / SIM_TIME:10.3f}")
    print(f"Server 2 utilization {AREA_SERVER_STATUS_2 / SIM_TIME:15.3f}")
    print(f"\nTime simulation ended {SIM_TIME:15.3f} minutes")


def update_time_avg_stats():
    """Prior to processing each event, this function updates the areas tracked
    for the calculation of continuous-time statistics."""
    global SIM_TIME, TIME_LAST_EVENT
    time_since_last_event = SIM_TIME - TIME_LAST_EVENT
    TIME_LAST_EVENT = SIM_TIME # current time

    # update area under num-in-queue function
    global AREA_NUM_IN_Q_1, Q1
    AREA_NUM_IN_Q_1 += len(Q1) * time_since_last_event
    global AREA_NUM_IN_Q_2, Q2
    AREA_NUM_IN_Q_2 += len(Q2) * time_since_last_event

    # update area under server-busy indicator function
    global AREA_SERVER_STATUS_1, AREA_SERVER_STATUS_2, SERVER_STATUS
    AREA_SERVER_STATUS_1 += SERVER_STATUS[0].value * time_since_last_event
    AREA_SERVER_STATUS_2 += SERVER_STATUS[1].value * time_since_last_event


def main():
    initialize()

    while True:
        timing()
        update_time_avg_stats()
        
        match NEXT_EVENT_TYPE:
            case 0:
                arrive1()
            case 1:
                arrive2()
            case 2:
                depart()
            case 3:
                report()
                break

main()



Average delay in queue 1       1.503 minutes
Average number in queue 1      1.515
Server 1 utilization           0.681

Average delay in queue 2       4.642 minutes
Average number in queue 2      4.667
Server 2 utilization           0.888

Time simulation ended        1000.000 minutes
